In [1]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")
rdd = rdd.map(lambda x: x.split(','))

In [2]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) != 0 and float(p[11]) != 0):
                return p

In [3]:
texilinesCorrect = rdd.filter(correctRows)

In [4]:
n = texilinesCorrect.count()

In [5]:
distance_money = texilinesCorrect.map(lambda x: (1, (float(x[5]), float(x[11]))))

In [6]:
distance_money.take(1)

[(1, (0.44, 3.5))]

In [6]:
def getSumPart(x):
            partOfDm = x[1][0] * (x[1][1] - (prev_m * x[1][0] + prev_b))
            partOfDb = x[1][1] - (prev_m * x[1][0] + prev_b)
            loss = (x[1][1] - (prev_m * x[1][0] + prev_b))**2
            return (x[0], (partOfDm, partOfDb, loss))

In [8]:
sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))

In [7]:
import numpy as np
import time 

In [18]:
def gradientDescent(learningRate = 0.00001, num_iteration = 100):
    prev_m = 00.1
    prev_b = 00.1
    count = 0
    #precision = 0.00000001 
    #prev_stepsize = 1
    while(num_iteration > count):
        start = time.time()
        # function to get inside sumation part for each formura
        # updated for every iteration.
        def getSumPart(x):
            partOfDm = - (x[1][0] * (x[1][1] - (prev_m * x[1][0] + prev_b)))
            partOfDb = -(x[1][1] - (prev_m * x[1][0] + prev_b))
            loss = (x[1][1] - (prev_m * x[1][0] + prev_b))**2
            return (x[0], (partOfDm, partOfDb, loss))
        # Get sumation part
        sumInGradient = distance_money.map(getSumPart)\
                            .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1], x1[2]+ x2[2]))\
                            .collect()
        # Calculate gradiant descent slope and intercept
        dm = (2.0/n) * sumInGradient[0][1][0]
        db = (2.0/n) * sumInGradient[0][1][1]
        # Updata slope and intercept
        cur_m = prev_m - (learningRate * dm)
        cur_b = prev_b - (learningRate * db)
        cur_loss = sumInGradient[0][1][2]

        print("Iteration: ", count)
        print("m: ", cur_m, "b: ", cur_b)
        print("Loss: ", cur_loss)

        count += 1

        # calculate previous step to compare with precison
        #prev_stepsize = np.mean([abs(prev_m - cur_m), abs(prev_b - cur_b)])

        prev_m = cur_m
        prev_b = cur_b

        stop = time.time()
        duration = stop - start
        print("Duration", duration)

    return cur_m, cur_b

In [19]:
m, b = gradientDescent()

Iteration:  0
m:  0.10127834308591331 b:  0.10022833924349964
Loss:  436320487.15051526
Duration 13.70647382736206
Iteration:  1
m:  0.10255615463698226 b:  0.10045659967105182
Loss:  435984937.35783815
Duration 12.957497835159302
Iteration:  2
m:  0.10383343487328174 b:  0.10068478131510558
Loss:  435649665.0854106
Duration 13.309316158294678
Iteration:  3
m:  0.10511018401479615 b:  0.10091288420809662
Loss:  435314670.103024
Duration 13.257608890533447
Iteration:  4
m:  0.10638640228141863 b:  0.10114090838244723
Loss:  434979952.18170583
Duration 13.57712197303772
Iteration:  5
m:  0.10766208989295141 b:  0.10136885387056628
Loss:  434645511.09169793
Duration 14.221048831939697


KeyboardInterrupt: 